In [5]:
import os
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import glob
import time

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Evaluating Resnet50 on tensorflow

#/content/drive/MyDrive/Documents/CIFAR-10-images-master/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
model = ResNet50(weights='imagenet')
img_path = '/content/drive/MyDrive/Documents/CIFAR-10-images-master/elephant.jpg'

label_names = {0: 'tench', 1: 'English springer', 2:'cassette player', 3:'chain saw', 4:'church', 5:'French horn', 6: 'garbage truck', 7:'gas pump', 8:'golf ball', 9:'parachute'}



#print(os.listdir(root))
#print([ name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name)) ])
def get_image_paths(root):
  
  count = 0
  index = 0
  image_paths = []
  labels = []

  for dir in os.listdir(root):
    os.chdir(root+'/'+dir)
    count += len(glob.glob('*.JPEG'))
    labels += [index] * len(glob.glob('*.JPEG'))

    for img in glob.glob('*.JPEG'):
      img_path = root+'/'+dir+'/'+img
      image_paths.append(img_path)
      #print(img_path)
  print(count)
  return image_paths

root = '/content/drive/MyDrive/Documents/CIFAR-10-images-master/test'
root = '/content/drive/MyDrive/Documents/imagenette2/val'

image_paths = get_image_paths(root)

t1 = time.time()
for img_path in image_paths:
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  preds = model.predict(x)
  print('Predicted:', decode_predictions(preds, top=3)[0])

print('Inference Time is: {} seconds'.format(time.time()-t1))

3925


KeyboardInterrupt: ignored

In [17]:
# Evaluating Resnet50 on pytorch

import torch
from torchvision import models
from PIL import Image
from torchvision import transforms


model = models.resnet50(pretrained=True)

root = '/content/drive/MyDrive/Documents/imagenette2/val'
image_paths = get_image_paths(root)

with open('/content/drive/MyDrive/Documents/CIFAR-10-images-master/imagenet1000_clsidx_to_labels.txt') as f:
  labels = [line.strip() for line in f.readlines()]


preprocess = transforms.Compose([
        #transforms.Resize(259),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(
        #mean=[0.485, 0.456, 0.406],
        #std=[0.229, 0.224, 0.225])
        ])

t1 = time.time()
for img in image_paths:
  #'/content/drive/MyDrive/Documents/CIFAR-10-images-master/ILSVRC2012_val_00037861.JPEG'
  ele = Image.open(img).convert('RGB')
  ele = preprocess(ele)
  ele = torch.unsqueeze(ele, 0)
  preds = model(ele)
  _, index = torch.max(preds, 1)
  percentage = torch.nn.functional.softmax(preds, dim=1)[0] * 1000
  print(labels[index[0]], percentage[index[0]].item())

#_, indices = torch.sort(preds, descending=True)
#[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

print('Inference Time is: {} seconds'.format(time.time()-t1))

3925
463: 'bucket, pail', 6.211338520050049
463: 'bucket, pail', 6.232949256896973
463: 'bucket, pail', 6.156910419464111
600: 'hook, claw', 5.777872562408447
731: "plunger, plumber's helper", 5.812431335449219
463: 'bucket, pail', 6.2333269119262695
463: 'bucket, pail', 6.069597244262695


KeyboardInterrupt: ignored

In [19]:
# Evaluating VGG16 on tensorflow

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
from tensorflow.keras.models import Model
import numpy as np

model = VGG16(weights='imagenet')

root = '/content/drive/MyDrive/Documents/imagenette2/val'
image_paths = get_image_paths(root)
# img_path = '/content/drive/MyDrive/Documents/CIFAR-10-images-master/elephant.jpg'
#x = np.expand_dims(x, axis=0)

t1 = time.time()
for img_path in image_paths:
  img = image.load_img(img_path, target_size=(224, 224))
  img = image.img_to_array(img)
  img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
  img = preprocess_input(img)
  
  labels = model.predict(img)
  label = decode_predictions(labels)
  print("the label is {} with {}% probability".format(label[0][0][1],label[0][0][2]*100))

print('Inference Time is: {} seconds'.format(time.time()-t1))

553467904/553467096 [==============================] - 4s 0us/step
3925
the label is tench with 67.58299469947815% probability
the label is tench with 90.43915867805481% probability
the label is tench with 99.7065007686615% probability
the label is tench with 99.99268054962158% probability
the label is tench with 99.96894598007202% probability
the label is tench with 99.65477585792542% probability
the label is tench with 99.21070337295532% probability
the label is tench with 87.67980337142944% probability
the label is tench with 99.04602766036987% probability
the label is sturgeon with 49.2327094078064% probability
the label is tench with 99.83640313148499% probability
the label is eel with 21.34491056203842% probability
the label is tench with 95.83162665367126% probability
the label is tench with 19.16365474462509% probability
the label is tench with 68.82497072219849% probability
the label is sturgeon with 42.97425448894501% probability
the label is tench with 99.99948740005493% pro

KeyboardInterrupt: ignored

In [21]:
# Evaluating VGG16 on pytorch

import torch
from torchvision import models
from PIL import Image
from torchvision import transforms

model = models.vgg16(pretrained=True)
preprocess = transforms.Compose([transforms.ToTensor(),])

with open('/content/drive/MyDrive/Documents/CIFAR-10-images-master/imagenet1000_clsidx_to_labels.txt') as f:
  labels = [line.strip() for line in f.readlines()]


root = '/content/drive/MyDrive/Documents/imagenette2/val'
image_paths = get_image_paths(root)


t1 = time.time()
for img in image_paths:
  #'/content/drive/MyDrive/Documents/CIFAR-10-images-master/ILSVRC2012_val_00037861.JPEG'
  ele = Image.open(img).convert('RGB')
  ele = preprocess(ele)
  ele = torch.unsqueeze(ele, 0)
  preds = model(ele)
  _, index = torch.max(preds, 1)
  percentage = torch.nn.functional.softmax(preds, dim=1)[0] * 100
  print(labels[index[0]], percentage[index[0]].item())





3925
{0: 'tench, Tinca tinca', 37.689659118652344
{0: 'tench, Tinca tinca', 53.64790725708008
669: 'mosquito net', 89.58068084716797
{0: 'tench, Tinca tinca', 68.7859878540039


KeyboardInterrupt: ignored